In [1]:
%config InlineBackend.figure_format = 'svg'
import ipywidgets as widgets
from IPython.display import display
import eq_functions as eq
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from datetime import datetime
from pyswarm import pso
from scipy import optimize

np.random.seed(1756)
random.seed(1756)

Read in data from text file. Store earthquakes as a list of 'Event' instances. The metrics dataframe contains location-specific data such as optimiser parameter bounds, latitude/longitude of injection well.

In [7]:
# set up dataframe for dataset/location specific metrics
rr = [42.088309, -113.387377] # 42.087593, -113.387119 ; 42.087905, -113.390624
nwb = [43.726077, -121.309651] #  43.726066, -121.310861 ; 43.726208, -121.309869 ; 43.726179, -121.309841 ; 43.726077, -121.309651
brd = [39.791999, -119.013119]
rrt0 = datetime(2010, 10, 2 , hour=8, minute=13, second=26) # first event times
nwbt0 = datetime(2012, 10, 29, hour=8, minute=2, second=21)
brdt0 = datetime(2010, 11, 13 , hour=8, minute=37, second=24)
metrics = pd.DataFrame({'lat0':[nwb[0], rr[0], brd[0]],
                        'long0':[nwb[1], rr[1], brd[1]],
                        't0':[nwbt0, rrt0, brdt0],
                        'ru':[10**2.63, 10**3.2, 1*1e3],
                        'rl':[3*1e1, 150, 1e1],
                        'T_inj':[1.901e+6, 3.416e+7, 1.642e+6],
                        'T_adjust':[432000, 0, 0],
                        'range':[slice(150,402),slice(82,168), slice(0,-1)],
                        'q_lwr':[10,28,10],
                        'q_upr':[1000,3469.95,1000],
                        'rc_lwr':[40,320,50],
                        'rc_upr':[100,450,300]},
    index =  ['newberry.txt','raft_river.txt','bradys.txt'])


def extract_events(fname):
# fname = 'newberry.txt'
    '''
    extract data from text file to store events in a list
    '''
    f = open(fname, 'r')
    flines = f.readlines()
    f.close()

    r = 6371e3 # earth radius in m
    lat0, long0, t0 =  metrics.loc[fname].lat0, metrics.loc[fname].long0, metrics.loc[fname].t0 #43.725820, -121.310371 OG # newberry # 42.088047, -113.385184# raft river #  
    x0 = eq.gnom_x(lat0,long0,lat0,long0)
    y0 = eq.gnom_y(lat0,long0,lat0,long0)

    # store events in list
    # latitudes and longitudes are projected to a plane using a gnomonic projection
    # distances between points and the origin (mean lat,lon) are determined by great circle distances between origin and the points
    events_all = [eq.Event(float(event.split()[10]), \
                           (datetime(int(event.split()[0]),int(event.split()[2]),int(event.split()[3]),hour=int(event.split()[4]),minute=int(event.split()[5]),second=int(float(event.split()[6])))-t0).total_seconds(), \
                           r*eq.gnom_x(float(event.split()[7]),float(event.split()[8]),lat0,long0, deg = True),\
                           r*eq.gnom_y(float(event.split()[7]),float(event.split()[8]),lat0,long0, deg = True),\
                           '-', eq.gcdist(r,float(event.split()[7]),lat0,float(event.split()[8]),long0, deg = True), 0) for event in flines]# if event.split()[0] in year]
    return events_all
    
loc_options = widgets.Select(options = ['newberry.txt','raft_river.txt','bradys.txt'], value = 'newberry.txt', description='Location')

fname = widgets.interact(lambda x: x, x=loc_options)

events_all = extract_events(fname)
type(events_all)

TypeError: expected str, bytes or os.PathLike object, not function

In [3]:
# put events into a catalog
catalog0 = pd.DataFrame({'Magnitude': [event.magnitude for event in events_all],
                                   'Time':[event.time for event in events_all],
                                   'x':[event.x for event in events_all],
                                   'y':[event.y for event in events_all],
                                   'Generation':[0] * len(events_all),
                                   'Distance_from_origin': [event.distance_from_origin for event in events_all],
                                   'Date':[datetime(int(event.split()[0]),int(event.split()[2]),int(event.split()[3]),hour=int(event.split()[4]),minute=int(event.split()[5]),second=int(float(event.split()[6]))) for event in flines]})
cols = ['Magnitude','Generation','x','y','Time','Distance_from_origin','Date']
catalog0 = catalog0.reindex(columns = cols)
catalog0 = catalog0[metrics.loc[fname].range] # get events from time period of interest
catalog0.Time = catalog0.Time - catalog0.Time.min() + metrics.loc[fname].T_adjust # shift time so that first event occurs however long after injection began 

TypeError: 'interactive' object is not iterable